Student Name: Tushara Govinda Reddy

Student ID:  220129510


Assignment 2, part 1 of 2
Questions 1 is a pen-and-paper exercise; questions 2-3 are coding exercises.

Please show your work - i.e., show and explain your code/math, and write your reasoning.

A data warehouse for a music streaming company consists of the dimensions song, user, time (time and date of when the user listened to a song), and a measure called fee (fee paid by the streaming company to the artist every time a user listens to that song).

[1 mark] Draw a schema diagram for the above data warehouse using a star schema.


[0.5 marks] Assume that the time dimension has 3 levels: day, month, year; and that the song and user dimensions both have 1 level (not including the virtual level 'all'). How many cuboids will this cube contain (including the base and apex cuboids)?

[0.5 marks] Starting with the base cuboid [time, user, song], what specific OLAP operations should be performed in order to list the total fee collected for a given song for a given month of a given year (e.g. October 2021)?

[1 mark] Using the same CSV file and data cube in the above lab tutorial, modify the "tutorial_model.json" file to include aggregate measures for the minimum and maximum amount in the data cube. Using these implemented aggregate measures, produce the values for the minimum and maximum amount in the data per year. Make sure to show your work in the PDF report. You can read the Cubes package documentation for assistance in this task.

In [ ]:
!pip install flask_sqlalchemy
!pip install cubes
!pip install -Iv sqlalchemy==1.3.9


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m/UNKNOWN
Value for scheme.scripts does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/bin

In [ ]:
from sqlalchemy import create_engine
from cubes.tutorial.sql import create_table_from_csv
from cubes import Workspace
from flask_sqlalchemy import SQLAlchemy

In [ ]:
engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine,
                      "IBRD_Balance_Sheet__FY2010.csv",
                      table_name="ibrd_balance",
                      fields=[
                          ("category", "string"),
                          ("category_label", "string"),
                          ("subcategory", "string"),
                          ("subcategory_label", "string"),
                          ("line_item", "string"),
                          ("year", "integer"),
                          ("amount", "integer")],
                      create_id=True
                     )

In [ ]:
workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")
workspace.list_cubes()
workspace.import_model("tutorial_model.json")
cube = workspace.cube("ibrd_balance")


In [ ]:
browser = workspace.browser(cube)
result = browser.aggregate()
result = browser.aggregate(aggregates=["amount_max"],drilldown=["year"])

for record in result:
  print(record)

{'year': 2009, 'amount_max': 110040}
{'year': 2010, 'amount_max': 128577}


In [ ]:
browser = workspace.browser(cube)
result = browser.aggregate()
result = browser.aggregate(aggregates=["amount_min"],drilldown=["year"])

for record in result:
  print(record)

{'year': 2009, 'amount_min': -1683}
{'year': 2010, 'amount_min': -3043}


Consider the CSV file "country-income-large.csv":

[1 marks] Load the CSV file using Cubes, create a JSON file for the data cube model, and create a data cube for the data. Use as dimensions the region, age, and online shopper fields. Use as measure the income. Define aggregate functions in the data cube model for the total, average, minimum, and maximum income. In your PDF report, show the relevant scripts and files you have created.

In [ ]:
engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine,
                      "country-income-large.csv",
                      table_name="country-income-large",
                      fields=[
                          ("Region", "string"),
                          ("Age", "integer"),
                          ("Income", "Integer"),
                          ("Online_Shopper", "string"),
                          ],
                      create_id=True
                     )

In [ ]:
workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")
workspace.list_cubes()
workspace.import_model("Assgn2_Part1.json")
cube = workspace.cube("country-income-large")

In [ ]:
browser = workspace.browser(cube)
result = browser.aggregate()
print("max value",result.summary["amount_max"])
print("min value",result.summary["amount_min"])
print("average value",result.summary["amount_average"])
print("max sum",result.summary["amount_sum"])


max value 143000
min value 31000
average value 85986.66666666667
max sum 5159200


[1 mark] Using the created data cube and data cube model, produce aggregate results for: the whole data cube; results per region; results per online shopping activity; and results for all people aged between 30 and 50.

In [ ]:
browser = workspace.browser(cube)
result = browser.aggregate()
result = browser.aggregate(drilldown=["Region"])
for record in result:
  print(record)

{'Region': 'Brazil', 'amount_sum': 1198200, 'record_count': 15, 'amount_max': 135000, 'amount_min': 31000, 'amount_average': 79880.0}
{'Region': 'India', 'amount_sum': 1954200, 'record_count': 23, 'amount_max': 141000, 'amount_min': 35000, 'amount_average': 84965.21739130435}
{'Region': 'USA', 'amount_sum': 2006800, 'record_count': 22, 'amount_max': 143000, 'amount_min': 45000, 'amount_average': 91218.18181818182}


In [ ]:
import cubes as cubes
browser = workspace.browser(cube)
result1 = browser.aggregate()
result1 = browser.aggregate(drilldown=["Online_Shopper"])
for record in result1:
  print(record)

{'Online_Shopper': 'No', 'amount_sum': 3150400, 'record_count': 36, 'amount_max': 143000, 'amount_min': 31000, 'amount_average': 87511.11111111111}
{'Online_Shopper': 'Yes', 'amount_sum': 2008800, 'record_count': 24, 'amount_max': 139000, 'amount_min': 41000, 'amount_average': 83700.0}


In [ ]:
import cubes as cubes
cuts = [cubes.RangeCut("Age", [30], [50])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell, drilldown=["Age"])
for record in result:
    print(record)

{'Age': 30, 'amount_sum': 59000, 'record_count': 1, 'amount_max': 59000, 'amount_min': 59000, 'amount_average': 59000.0}
{'Age': 32, 'amount_sum': 57600, 'record_count': 1, 'amount_max': 57600, 'amount_min': 57600, 'amount_average': 57600.0}
{'Age': 34, 'amount_sum': 58000, 'record_count': 1, 'amount_max': 58000, 'amount_min': 58000, 'amount_average': 58000.0}
{'Age': 35, 'amount_sum': 120800, 'record_count': 2, 'amount_max': 64800, 'amount_min': 56000, 'amount_average': 60400.0}
{'Age': 36, 'amount_sum': 70000, 'record_count': 1, 'amount_max': 70000, 'amount_min': 70000, 'amount_average': 70000.0}
{'Age': 39, 'amount_sum': 71000, 'record_count': 1, 'amount_max': 71000, 'amount_min': 71000, 'amount_average': 71000.0}
{'Age': 40, 'amount_sum': 69600, 'record_count': 1, 'amount_max': 69600, 'amount_min': 69600, 'amount_average': 69600.0}
{'Age': 41, 'amount_sum': 64000, 'record_count': 1, 'amount_max': 64000, 'amount_min': 64000, 'amount_average': 64000.0}
{'Age': 42, 'amount_sum': 22940